In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features

df_train = pd.read_csv('./data/train_filtrado.csv')
df_train['precio'] = np.log(df_train['precio'])
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train['precio'] = np.log(df_train['precio'])

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [2]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

## LightGBM model

In [3]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.08363501292068126)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.8988911725316586,
 'bagging_freq': 22.0,
 'feature_fraction': 0.6622442122619671,
 'learning_rate': 0.16422725363286422,
 'max_depth': 22.0,
 'num_leaves': 180.0,
 'test_size': 0.20892455926004772}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [4]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

## XGBoost

In [5]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [7]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer

features = ['habitaciones', 
            'garages', 
            'banos',
            'antiguedad',
            'metroscubiertos', 
            'metrostotales',
            'lat_norm', 'lng_norm'
            'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
                 'promedio_precio_hbg_tipo_propiedad']

features += features_test

features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
                 'promedio_precio_hbg_tipo_propiedad']

features += features_test

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('lightgbm', lgb_m), 
#            ('keras', keras_m), 
           ('xgboost', xgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [lightgbm: LightGBMWrapper]
    fold  0:  [0.20419658]
    fold  1:  [0.20448883]
    fold  2:  [0.20467907]
    fold  3:  [0.20347208]
    ----
    MEAN:     [0.20420914] + [0.00045893]

estimator  1: [xgboost: XGBoostWrapper]
[18:01:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:11.9652
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:10.2902
[2]	validation_0-mae:8.84998
[3]	validation_0-mae:7.6111
[4]	validation_0-mae:6.54574
[5]	validation_0-mae:5.62954
[6]	validation_0-mae:4.84163
[7]	validation_0-mae:4.164
[8]	validation_0-mae:3.58128
[9]	validation_0-mae:3.08017
[10]	validation_0-mae:2.64919
[11]	validation_0-mae:2.2786
[12]	validation_0-mae:1.95995
[13]	validation_0-mae:1.68608
[14]	validation_0-mae:1.45074
[15]	validation_0-mae:1.24861
[1

[92]	validation_0-mae:0.066138
[93]	validation_0-mae:0.0659
[94]	validation_0-mae:0.065614
[95]	validation_0-mae:0.065119
[96]	validation_0-mae:0.064805
[97]	validation_0-mae:0.064612
[98]	validation_0-mae:0.063859
[99]	validation_0-mae:0.063547
[100]	validation_0-mae:0.063049
[101]	validation_0-mae:0.062387
[102]	validation_0-mae:0.061902
[103]	validation_0-mae:0.061447
[104]	validation_0-mae:0.060543
[105]	validation_0-mae:0.060296
[106]	validation_0-mae:0.05957
[107]	validation_0-mae:0.058843
[108]	validation_0-mae:0.0584
[109]	validation_0-mae:0.058186
[110]	validation_0-mae:0.057874
[111]	validation_0-mae:0.057553
[112]	validation_0-mae:0.057483
[113]	validation_0-mae:0.057082
[114]	validation_0-mae:0.05688
[115]	validation_0-mae:0.056533
[116]	validation_0-mae:0.055916
[117]	validation_0-mae:0.055684
[118]	validation_0-mae:0.055259
[119]	validation_0-mae:0.054773
[120]	validation_0-mae:0.054587
[121]	validation_0-mae:0.054225
[122]	validation_0-mae:0.053953
[123]	validation_0-mae

[42]	validation_0-mae:0.121122
[43]	validation_0-mae:0.118587
[44]	validation_0-mae:0.116832
[45]	validation_0-mae:0.114146
[46]	validation_0-mae:0.11276
[47]	validation_0-mae:0.110885
[48]	validation_0-mae:0.108986
[49]	validation_0-mae:0.107538
[50]	validation_0-mae:0.105899
[51]	validation_0-mae:0.104058
[52]	validation_0-mae:0.102569
[53]	validation_0-mae:0.100944
[54]	validation_0-mae:0.100016
[55]	validation_0-mae:0.098213
[56]	validation_0-mae:0.0971
[57]	validation_0-mae:0.095917
[58]	validation_0-mae:0.095005
[59]	validation_0-mae:0.094389
[60]	validation_0-mae:0.093215
[61]	validation_0-mae:0.092257
[62]	validation_0-mae:0.09081
[63]	validation_0-mae:0.090192
[64]	validation_0-mae:0.088683
[65]	validation_0-mae:0.087276
[66]	validation_0-mae:0.086841
[67]	validation_0-mae:0.085545
[68]	validation_0-mae:0.084438
[69]	validation_0-mae:0.083734
[70]	validation_0-mae:0.082638
[71]	validation_0-mae:0.081776
[72]	validation_0-mae:0.080875
[73]	validation_0-mae:0.08033
[74]	validati

In [8]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [14]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [15]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [16]:

hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m_2nd = XGBoostWrapper(**hps)

xgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

[18:27:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:11.963
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:10.2883
[2]	validation_0-mae:8.84807
[3]	validation_0-mae:7.60942
[4]	validation_0-mae:6.54419
[5]	validation_0-mae:5.62819
[6]	validation_0-mae:4.84034
[7]	validation_0-mae:4.16281
[8]	validation_0-mae:3.58011
[9]	validation_0-mae:3.07896
[10]	validation_0-mae:2.64792
[11]	validation_0-mae:2.27724
[12]	validation_0-mae:1.95845
[13]	validation_0-mae:1.68436
[14]	validation_0-mae:1.44858
[15]	validation_0-mae:1.24591
[16]	validation_0-mae:1.0715
[17]	validation_0-mae:0.921609
[18]	validation_0-mae:0.792701
[19]	validation_0-mae:0.681765
[20]	validation_0-mae:0.58641
[21]	validation_0-mae:0.504503
[22]	validation_0-mae:0.434236
[23]	validation_0-mae:0.373886
[24]	validation_0-mae:0.322083
[25]	validation_0-mae:0.277377
[26]	validation_0-mae:0.23902
[27]

XGBoostWrapper(alpha=20.91434940058063, base_score=0.5, booster='gbtree',
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.65,
               early_stopping_rounds=5, gamma=0, importance_type='gain',
               learning_rate=0.14, max_delta_step=0, max_depth=16,
               min_child_weight=1, missing=None, n_estimators=150, n_jobs=4,
               nthread=None, objective='reg:linear', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, test_size=0.2, verbosity=1)

In [17]:
df_test_s['target'] = np.exp(xgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02'])))
df_test_s[['id', 'target']].to_csv('respuesta29.csv', index = False)

## Prediccion solo con features de stacking

In [17]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features)), df_train_f['precio'].values)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



LightGBMWrapper(bagging_fraction=0.8924398062087346, bagging_freq=36,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.16167385124183287, importance_type='split',
                learning_rate=0.054693418899570134, max_depth=4,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=93,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [24]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

MAE St

In [26]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta21.csv', index = False)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



In [13]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

 12%|█▏        | 12/100 [00:48<05:53,  4.02s/it, best loss: 437714.00137358136]


KeyboardInterrupt: 

In [141]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")

MAE Keras (train): 524925.45271
